In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
import pandas as pd

stores_df = pd.read_excel("https://data.gov.au/data/dataset/2fe5e2a9-8a3d-4dcf-baec-c5147d953150/resource/b59a15df-86ea-4c4c-95be-4dd9fc9f8ac4/download/2019-20-historical-migration-statistics-locked.xlsx", sheet_name="1.2", header=7 )
# stores_df = pd.read_excel("datasets/2019-20-historical-migration-statistics-locked.xlsx", sheet_name="1.2", header=7 )

stores_df = stores_df.iloc[: , :-1]
stores_df = stores_df.iloc[: , 1:]
stores_df = stores_df.melt(id_vars=["Year"], 
        var_name="country", 
        value_name="migration_no")
stores_df.rename(index=str, columns={"Year": "year"}, inplace=True)

stores_df.head()
